In [2]:
# These are helpful routines that will assist in building this book. 
# You should run this block before anything else. There is no output expected.
from astrodynamicsbook.bookhelpers import *
loadLatexPreamble()

# This is only here to create the navigation link:
genPrevLink()

$
\def\bs{\boldsymbol}
\def\mf{\mathbf}
\def\mb{\mathbb}
\def\mc{\mathcal}
\def\rfr{\mathcal}
\def\grad{{\rm grad}}
\def\Re{{\rm Re}}
\def\Im{{\rm Im}}
\def\und{\underline}
\def\ovl{\overline}
\def\unb{\underbrace}
\def\Log{\mbox{Log}}
\def\bfomega{\bs \omega}
\def\bfalpha{\bs \alpha}
\def\da{\triangleq}
\newcommand{\leftexp}[2]{{\vphantom{#2}}^{#1}\!{#2}}
\newcommand{\leftsub}[2]{{\vphantom{#2}}_{#1}\!{#2}}
\newcommand{\omegarot}[2]{{\leftexp{\mathcal{#1}}{\boldsymbol{\omega}}^{\mathcal{#2}}}}
\newcommand{\alpharot}[2]{{\leftexp{\mathcal{#1}}{\boldsymbol{\alpha}}^{\mathcal{#2}}}}
\newcommand{\framerot}[2]{{\leftexp{\mathcal{#1}}{C}^{\mathcal{#2}}}}
\newcommand{\dframerot}[2]{{\vphantom{\dot{C}}^{\mathcal{#1}}\!{\dot{C}^{\mathcal{#2}}}}}
\newcommand{\bdot}[1]{\dot{\mathbf{#1}}}
\newcommand{\bhat}[1]{\hat{\mathbf{#1}}}
\newcommand{\mbhat}[1]{\hat{\mathbb{#1}}}
\def\iwb{\omegarot{I}{B}}
\def\iab{\alpharot{I}{B}}
\def\icb{\framerot{I}{B}}
\def\dif{\mathop{}\!\mathrm{d}}
\newcommand{\intd}[1]{\dif#1}
\newcommand{\od}[3][]{{ \frac{\dif{^{#1}}#2}{\dif{#3^{#1}}} }}			
\newcommand{\pd}[3][]{{ \frac{\partial{^{#1}}#2}{\partial{#3^{#1}}} }}	 
\newcommand{\md}[6]{{  \frac{\partial{^{#2}}#1}{\partial{#3^{#4}}\partial{#5^{#6}}} }}
\newcommand{\fddt}[2][]{{  \leftexp{\mathcal{#2}}{\frac{\dif{#1}}{\dif{t}}}  }}
\newcommand{\fdddt}[2][]{{  \leftexp{\mathcal{#2}}{\frac{\dif{^{2}#1}}{\dif{t^2}}}  }}
\newcommand{\ddt}[1][]{\fddt[#1]{I}}
$


# [Previous](<20-Series Solutions for f and g.ipynb>)

# A (Minor) Annoyance

While we have now provided ourselves with methods for propagating any type of orbit corresponding to any of the open or closed conic sections, there's an extra bit of processing that constantly has to happen as we check on the type of orbit we're dealing with, and determine which anomaly we should be calculating, and which equations we should be applying.  This isn't really a huge deal, but it does tend to make code more complicated, and more complex code invariably means buggier code.  More importantly, it's not *elegant*. We have a completely unified description of *all* orbit types via the orbital state vectors.  Why can't we have a unified description in orbital elements?

<div class="alert alert-block alert-danger">
    Never underestimate the importance of the dual principles of parsimony and elegance in science and engineering.  There's not only beauty, but extreme utility in simplicity.  Simple solutions should always be preferred to more complex ones because simple solutions are so much less likely to fail in new and exciting ways, just when you least expect them to.  Note, however, that 'simple' is not synonymous with 'brief', especially when it comes to code.  If your incredibly clever one-liner replaces a dozen lines of code but is incomprehensible to anyone but you, then it is not the more elegant solution.  When using highly idiomatic languages like Python, elegance resides in utilizing the language's standard idioms, and eschewing obscure and confuscated approaches.
</div>

# A Universal Variable

Turns out, it is entirely possible to define a single auxiliary variable (in place of the eccentric, parabolic, and hyperbolic anomalies) and to create a (mostly) unified orbital element description of all orbits.  Getting there, however, is not entirely straightforward, but fortunately this is well-trod ground.  In particular, we will be following in the footsteps of Battin, and adopting much of the nomenclature and conventions from his various articles and books.

We start by returning to the [orbital specific energy](<14-Keplers Laws Continued.ipynb#Effective-Potential>):
$$\mc E = \frac{v^2}{2} - \frac{\mu}{r} = -\frac{\mu}{2a} = \frac{\dot r^2}{2} - \frac{\mu}{r} +  \frac{h^2}{2r^2} $$

Solving the last two expressions for $\dot r^2$, we have:
$$\dot r^2 = \frac{2\mu}{r} - \frac{h^2}{r^2} - \frac{\mu}{a} = \frac{2\mu}{r} - \frac{\mu\ell}{r^2} - \frac{\mu}{a} $$
where we substituted $h = \sqrt{\mu\ell}$ in the final expression.

We now define our universal variable $\chi$ via its derivative, such that:
<div class="alert alert-block alert-info">
    $$\dot\chi \triangleq \frac{\sqrt{\mu}}{r}$$
</div>

Let's consider the expression:
$$ \left(\frac{\dot r}{\chi}\right)^2 = \left(\frac{ \dfrac{\intd{r}}{\intd{t}} }{\dfrac{\intd{\chi}}{\intd{t}} }\right)^2 = \left(\dfrac{\intd{r}}{\intd{\chi}}\right)^2 $$

Substituting in the expression we derived above for $\dot r^2$ and our definition for $\dot \chi$, we thus have:
$$\left(\dfrac{\intd{r}}{\intd{\chi}}\right)^2  = \left(\frac{2\mu}{r} - \frac{\mu\ell}{r^2} - \frac{\mu}{a}\right) \left(\frac{r^2}{\mu}\right) = 2r - \ell - \frac{r^2}{a}$$

Taking the square root of the whole expression and separating variables, we have:
$$\intd{\chi} = \left( 2r - \ell - \frac{r^2}{a}\right)^{-\frac{1}{2}} \intd{r} \quad \Longrightarrow \quad \int \intd{\chi} = \sqrt{a} \int \frac{\intd{r}}{\sqrt{2 a r - a l - r^{2}}}$$

At this point, we reach for our handy table of integrals to find that:

$$ \int \frac{\intd{x}}{\sqrt{c^2 - x^{2}}} = \sin^{-1}\left(\frac{x}{c}\right)$$

<div class="alert alert-block alert-danger">
    What? You don't have a handy table of integrals?  Shame on you.  Print one out immediately and put it wherever you stuck your table of trigonometric identities. Better yet, don't waste paper but always know where you can access one. 
</div>

Let's let $x = 2\left(1 - \dfrac{r}{a}\right)$. Then: $\intd{x} = -\dfrac{2}{a}\intd{r}$ and so:
$$\frac{\intd{r}}{\sqrt{2 a r - a l - r^{2}}} = - \frac{\intd{x}}{\sqrt{4\left(1 - \dfrac{l}{a}\right) - x^{2}}} $$
meaning that in this case $c = 4\left(1 - \dfrac{l}{a}\right)$ and the integration works out to:
$$ \chi + c_0 = - \sqrt{a}\sin^{-1}\left( \frac{2\left(1 - \dfrac{r}{a}\right)}{\sqrt{4\left(1 - \dfrac{l}{a}\right)}}\right) = \sqrt{a}\sin^{-1}\left( \frac{\left(\dfrac{r}{a}-1\right)}{e}\right)$$
where $c_0$ is our constant of integration and we substituted $\ell = a(1-e^2)$. 

At this point, we can solve for $r$ as a function of $\chi$ and write:
$$r = a\left(1 + e\sin\left(\frac{\chi + c_o}{\sqrt{a}} \right)\right)$$

## Negative Semi-Major Axes
This is great and all, but there's a slight problem here.  In order to have carried out this integration, we were forced to assume that $a$ is positive (did you catch it - it happened when we moved the $\sqrt{a}$ term outside of the integrand).  By inspection, we see that this expression for $r$ doesn't make sense for a negative $a$, which would lead to imaginary orbital radii. 

We set out to find a unified description of all orbits, but on practically our first step, we found ourselves assuming a positive semi-major axis, thus producing expressions valid for only closed orbits.  However, we will be saved here by one super convenient fact:

$$ \int \frac{\intd{x}}{\sqrt{c^2 + x^{2}}} = \sinh^{-1}\left(\frac{x}{c}\right)$$

We have already seen that the trigonometric and hyperbolic functions operate in very similar fashions, with basically the same identities. If we can show that we get an equivalent $\sinh$ form for negative $a$, then everything we figure out under the positive $a$ assumption should still work for negative $a$ values. So let's see what happens when we assume that $a$ is strictly negative.  Just to make things clearer, we'll define $a_n \triangleq -a$, such that $a_n$ is positive for negative $a$.  We thus have:
$$\intd{\chi} = \left( 2r - \ell + \frac{r^2}{a_n}\right)^{-\frac{1}{2}} \intd{r} \quad \Longrightarrow \quad \int \intd{\chi} = \sqrt{a_n} \int \frac{\intd{r}}{\sqrt{2 a_n r - a_n l + r^{2}}}$$

In this case, we'll take $x = 2\left(1 + \dfrac{r}{a_n}\right)$, such that $\intd{x} = \dfrac{2}{a_n}\intd{r}$ and:
$$\frac{\intd{r}}{\sqrt{2 a_n r - a_n l + r^{2}}} = \frac{\intd{x}}{\sqrt{-4\left(1 + \dfrac{l}{a_n}\right) + x^{2}}} $$

So, for the hyperbolic case, we have:
$$ \chi + c_0 = \sqrt{a_n}\sinh^{-1}\left( \frac{2\left(1 + \dfrac{r}{a_n}\right)}{\sqrt{-4\left(1 + \dfrac{l}{a_n}\right)}}\right) = \sqrt{-a}\sinh^{-1}\left( \frac{\left(1 - \dfrac{r}{a}\right)}{e}\right)$$
As with our previous treatments of hyperbolic orbits, we see that we get exactly the same forms as in the elliptical case, save for use of hyperbolic functions and sign flips.  We will thus proceed with the rest of our derivation using the positive $a$ assumption, knowing that we can readily adapt our results to the hyperbolic case without too much extra work.

# Universal Variable Initial Conditions

Returning to our orbital radius expression as a function of $\chi$, we now plug this back into our original definition of $\dot \chi$ to get:
$$\dot\chi \triangleq \frac{\sqrt{\mu}}{r} = \frac{\sqrt{\mu}}{a\left(1 + e\sin\left(\frac{\chi + c_o}{\sqrt{a}} \right)\right)}$$

Again separating variables, we have:
$$\intd{\chi}\left(a\left(1 + e\sin\left(\frac{\chi + c_o}{\sqrt{a}} \right)\right) \right) = \intd{t}\sqrt{\mu}$$
which we integrate to find:

$$a \left.\left( \chi - \sqrt{a}e \cos\left( \frac{\chi + c_o}{\sqrt{a}} \right) \right)\right\vert_{\chi(t_0)}^{\chi(t)} = \sqrt{\mu}\underbrace{t - t_0}_{\displaystyle \triangleq \Delta{t}}$$

Without loss of generality, we can assume that $\chi(t_0) \equiv 0$, as $\chi$ is an arbitrary auxiliary variable and can be associated with time in any way we see fit. For notational convenience, we will henceforth assume that $\chi$ represents $\chi(t)$. Expanding out our integrated expression (and utilizing the cosine angle addition identity), we now have:

$$ \sqrt{\mu}\Delta{t} = a\chi - a^{\frac{3}{2}}e\left(\underbrace{\cos\left(\frac{\chi}{\sqrt{a}}\right)\cos\left(\frac{c_0}{\sqrt{a}}\right) - \sin\left(\frac{\chi}{\sqrt{a}}\right)\sin\left(\frac{c_0}{\sqrt{a}}\right)}_{\displaystyle \equiv \cos\left(\frac{\chi + c_0}{\sqrt{a}}\right)} - \cos\left(\frac{c_0}{\sqrt{a}}\right)\right) $$

Returning to our radius expression, we can define $r_0$ as the radius at $t_0$:

$$r_0 \triangleq r(t_0) =  a\left(1 + e\sin\left(\frac{c_o}{\sqrt{a}} \right)\right) \quad \Longrightarrow \quad e\sin\left(\frac{c_o}{\sqrt{a}}\right) = \frac{r_0}{a} - 1$$

where we have once again taken advantage of our setting $\chi(t_0)$ to be zero.

Differentiating the radius expression:

$$\dot r = \frac{ae}{\sqrt{a}} \dot\chi \cos\left(\frac{\chi + c_0}{\sqrt{a}}\right)\quad \Longrightarrow \quad  \frac{r\dot{r}}{\sqrt{\mu a}} = e \cos\left(\frac{\chi + c_0}{\sqrt{a}}\right)$$

where we substituted in $\sqrt{\mu}/r$ for $\dot\chi$.

Recall that a byproduct of our derivation of [Lagrange's Fundamental Invarianats](<20-Series Solutions for f and g.ipynb#Lagrange's-Fundamental-Invariants>) was finding that $\mf r \cdot \mf v = r \dot{r}$, meaning that for initial orbit state vectors $\mf r_0 \triangleq \mf r(t_0)$ and $\mf v_0 \triangleq \mf v(t_0)$ we have:

$$e \cos\left(\frac{c_0}{\sqrt{a}}\right) = \frac{\mf r_0 \cdot \mf v_0}{\sqrt{\mu a}}$$

Note that we now have expression for both the cosines and sine terms in $c_0$ appearing in our expanded, integrated equation for $\sqrt{\mu}\Delta{t}$, above.  Note that each is also scaled by eccentricity, which means that we can eliminate both $c_0$ and the eccentricity from this expression entirely by substituting in these new equations.  Let's do so and see where we're at:

$$\sqrt{\mu}\Delta{t} = a\chi - a^{\frac{3}{2}}\left(\cos\left(\frac{\chi}{\sqrt{a}}\right)\left( \frac{\mf r_0 \cdot \mf v_0}{\sqrt{\mu a}}\right) - \sin\left(\frac{\chi}{\sqrt{a}}\right)\left(\frac{r_0}{a} - 1\right) -\left( \frac{\mf r_0 \cdot \mf v_0}{\sqrt{\mu a}}\right)\right)$$

We can similarly eliminate $c_0$ from our $r$ expression by first expanding and then substituting:
$$\begin{split}
r &= a\left(1 + e\sin\left(\frac{\chi + c_o}{\sqrt{a}} \right)\right) = a + ae\left(\sin\left(\frac{\chi}{\sqrt{a}} \right)\cos\left(\frac{c_0}{\sqrt{a}}\right)+ \sin\left(\frac{c_0}{\sqrt{a}} \right)\cos\left(\frac{\chi}{\sqrt{a}}\right)\right)\\
&= a + a\left(\sin\left(\frac{\chi}{\sqrt{a}} \right)\left( \frac{\mf r_0 \cdot \mf v_0}{\sqrt{\mu a}}\right) + \left(\frac{r_0}{a} - 1\right)\cos\left(\frac{\chi}{\sqrt{a}}\right)\right)
\end{split}
$$


# Universal Variable Unification

Let's regroup.  At this point we have an expression for the orbital radius magnitude at a time $t$ (which, as usual, we can relate to some initial time $t_0$ as $t = t_0 + \Delta{t}$) as a function of $\chi$, the orbital state vectors at $t_0$, and the gravitational parameter and semi-major axis.  We similarly have an expression for $\Delta{t}$ as a function of all of the same quantities.  All that's left for us to do in order to propagate our orbits using the universal variable formulation is to actually find $\chi(t)$.

To help us along, we define a new variable:

$$ \psi \triangleq \frac{\chi^2}{a} $$

noting that $\psi$ will be positive for closed orbits and negative for open orbits (and zero for parabolae). Substituting into our equations, we now have:

$$\begin{split}
r& = \chi^2\left(\frac{1 - \cos\left(\sqrt{\psi}\right)}{\psi}\right) + \left( \frac{\mf r_0 \cdot \mf v_0}{\sqrt{\mu}}\right)\chi\left(1 - \psi \frac{\sqrt{\psi} - \sin\left(\sqrt{\psi}\right)}{\sqrt{\psi^3}}\right) + r_0\left( 1 - \psi \frac{\sqrt{\psi} - \cos\left(\sqrt{\psi}\right)}{\sqrt{\psi}}\right)\\
\sqrt{\mu}\Delta{t} &= \chi^3 \left( \frac{\sqrt{\psi} - \sin\left(\sqrt{\psi}\right)}{\sqrt{\psi^3}}\right) + \left( \frac{\mf r_0 \cdot \mf v_0}{\sqrt{\mu}}\right)\chi^2 \left(\frac{1 - \cos\left(\sqrt{\psi}\right)}{\psi}\right) + r_0 \chi\left(1 - \psi \frac{\sqrt{\psi} - \sin\left(\sqrt{\psi}\right)}{\sqrt{\psi^3}}\right)
\end{split}$$

We see that these two expression are built out of the same basic (and frequently repeated) blocks.  Two, in particular, stand out:
$$\frac{\sqrt{\psi} - \sin\left(\sqrt{\psi}\right)}{\sqrt{\psi^3}} \quad \textrm{and} \quad \frac{1 - \cos\left(\sqrt{\psi}\right)}{\psi}$$

An incredible thing happens if we go back and re-derive these two equations under the assumption of a negative $a$. We get exactly the same combinations of terms, save that these two blocks have the forms:
$$\frac{\sinh\left(\sqrt{-\psi}\right) - \sqrt{-\psi}}{\sqrt{-\psi^3}} \quad \textrm{and} \quad \frac{1 - \cosh\left(\sqrt{-\psi}\right)}{\psi}$$


<div class="alert alert-block alert-warning">
<b>Exercise</b><br> Trust! But Verify!
</div>

Here, finally, is our unified universal form.  

<div class="alert alert-block alert-info">
We define:

$$c_2 \triangleq \begin{cases}
\displaystyle \frac{1 - \cos\left(\sqrt{\psi}\right)}{\psi} & \psi \ge 0\\
\displaystyle  \frac{1 - \cosh\left(\sqrt{-\psi}\right)}{\psi} & \psi < 0
\end{cases}
\qquad 
c_3 \triangleq \begin{cases}
\displaystyle \frac{\sqrt{\psi} - \sin\left(\sqrt{\psi}\right)}{\sqrt{\psi}^3} & \psi \ge 0\\
\displaystyle \frac{ \sinh\left(\sqrt{-\psi}\right) - \sqrt{-\psi} }{\sqrt{-\psi}^3} & \psi < 0
\end{cases}$$

such that:

$$ \begin{split}
r &= \chi^2 c_2 + \frac{\mf r_0 \cdot \mf v_0}{\sqrt{\mu}}\chi(1 - \psi c_3)  + r_0\left(1 - \psi c_2\right)\\
n\Delta t &= c _3\left( \frac{\chi }{\sqrt{a}} \right)^3 +  c_2 \frac{\mf r_0 \cdot \mf v_0}{\sqrt{\mu a}}\left( \frac{\chi}{\sqrt{a}}\right)^2 + \frac{r_o}{a}\left( \frac{\chi}{\sqrt{a}}\right)(1 - \psi c_3)
\end{split}$$
    </div>
    
where we substituted $\sqrt{\mu} = n\sqrt{a^3}$ in the final expression.

# Universal Variable Propagation

All that's left for us to do now is to figure out how to compute $\chi$ for a given $\Delta t$.  Note that nothing we've done here has mitigated the transcendental nature of the time equation (and nothing ever will). We have hidden away the various trigonometric terms that make our lives difficult, but they're all still there, and there is no way to directly invert the equation for $n\Delta{t}$ to solve for $\chi$.

But that's okay.  We've already provided ourself with a robust method for inverting the time equation via Newton-Raphson iteration, and we can once again return to this well-trod ground here. We go back to our fundamental definition of the universal variable:
$$ \frac{\intd{\chi}}{\intd{t}} = \frac{\sqrt{\mu}}{r} \quad \Longrightarrow\quad \sqrt{\mu}\intd{t} = r\intd{\chi}\quad \Longrightarrow\quad \sqrt{\mu}\Delta{t} = \int r\intd{\chi}$$

We thus take our minimization function to be:
$$ f(\chi) = \sqrt{\mu}\Delta{t} - \int r\intd{\chi} \quad  \Longrightarrow\quad f'(\chi) =\frac{\intd{f}}{\intd{\chi}} = -r$$
with the iterant defined as:
$$ \chi_{n+1} = \chi_n - \frac{f(\chi_n)}{f'(\chi_n)}$$

Note that we have already evaluated the integral of r with respect to $\chi$, above, when finding our original expression for $\sqrt{\mu}\Delta{t}$.  Returning to this result, we can write out the full form of the iterant as:

<div class="alert alert-block alert-info">
$$ \chi_{n+1} = \chi_n + \frac{1}{r}\left( \sqrt{\mu}\Delta t - \chi_n^3 c_3 - \frac{\mf r_0 \cdot \mf v_0}{\sqrt{\mu}} \chi_n^2 c_2 - r_0 \chi_n(1 - \psi c_3)  \right) $$   
    </div>

## Universal Variable Newton-Raphson Initialization

As always, there is the question of selecting initial conditions. Let's return to our first expression for $r$ (remembering that it carries the implicit assumption that $a > 0$):
$$r = a\left(1 + e\sin\left(\frac{\chi + c_o}{\sqrt{a}} \right)\right) = a(1-e\cos(E))$$

Because this form of $r$ assumes a closed orbit, we can equate it to the equivalent orbital radius expression as a function of eccentric anomaly.  The equality simplifies as:
$$-e\cos(E) = e\sin\left(\frac{\chi + c_o}{\sqrt{a}} \right) = -e\cos\left(\frac{\pi}{2} + \frac{\chi + c_o}{\sqrt{a}}\right)$$
where we have taken advantage of the fact that cosine and sine are exactly phase shifted by 90$^\circ$.  Thus:
$$E = \frac{\pi}{2} + \frac{\chi + c_o}{\sqrt{a}}$$
and
$$\Delta{E} = E - E_0 = \frac{\pi}{2} + \frac{\chi + c_o}{\sqrt{a}} - \left(\frac{\pi}{2} + \frac{\chi(t_0) + c_o}{\sqrt{a}}\right)$$

Recalling that we set $\chi(t_0) \equiv 0$, this means that
$$\Delta{E} = \frac{\chi}{\sqrt{a}}$$

Since the hyperbolic equations end up doing exactly what their trigonometric counterparts do, this also tells us that:
$$\Delta{H} = \frac{\chi}{\sqrt{-a}}$$

For the parabolic case, we have $a = \infty$ which means that $\psi = 0$.

<div class="alert alert-block alert-danger">
  At this point, you might be thinking that $c_2$ and $c_3$ are undefined for $\psi = 0$.  Nope!  You've forgotten about L'Hôpital's rule (or just Taylor Expansion).  Both the numerator and denominator of $c_2$ and $c_3$ evaluate to zero for $\psi = 0$, and so their value is the limit of the derivative of the numerator divided by the derivative of the denominator as $\psi \to 0$.  Working this out, we find that for $\psi = 0$, $c_2 = \dfrac{1}{2}$ and $c_3 = \dfrac{1}{6}$.  Go ahead and check to make sure you agree.
</div>

We thus have:
$$ r =  \chi^2 c_2 + \frac{\mf r_0 \cdot \mf v_0}{\sqrt{\mu}}\chi(1 - \psi c_3)  + r_0\left(1 - \psi c_2\right) = $$